<a href="https://colab.research.google.com/github/jgillet4/hackWestern7/blob/master/NLP_MachineLearning_StoryBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NLP Processing with N-gram, Model 1

In [113]:
# >>> import nltk
# >>> nltk.download('reuters')
# >>> nltk.download('punkt')

from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

# Create a placeholder for model
model = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of co-occurance  
for sentence in reuters.sents():
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        model[(w1, w2)][w3] += 1
 
# Let's transform the counts to probabilities
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

print("hello")

hello


In [114]:
#use the above model to predict for two words, the next word
dict(model["today","the"])

{'Bank': 0.05555555555555555,
 'European': 0.05555555555555555,
 'Higher': 0.05555555555555555,
 'Italian': 0.05555555555555555,
 'Turkish': 0.05555555555555555,
 'company': 0.16666666666666666,
 'emirate': 0.05555555555555555,
 'increase': 0.05555555555555555,
 'newspaper': 0.05555555555555555,
 'options': 0.05555555555555555,
 'overseas': 0.05555555555555555,
 'pound': 0.05555555555555555,
 'price': 0.1111111111111111,
 'public': 0.05555555555555555,
 'time': 0.05555555555555555}

# Creating full sentences

In [115]:
import random

# starting words
text = ["today", "the"]
sentence_finished = False
 
while not sentence_finished:
  # select a random probability threshold  
  r = random.random()
  accumulator = .0

  for word in model[tuple(text[-2:])].keys():
      accumulator += model[tuple(text[-2:])][word]
      # select words that are above the probability threshold
      if accumulator >= r:
          text.append(word)
          break

  if text[-2:] == [None, None]:
      sentence_finished = True
 
print (' '.join([t for t in text if t]))


today the price of Encor shares as a major wheat producing countries , but said it will be " repugnant ."


# Building a neural network model that is more powerful than N-gram

In [117]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [74]:
data_text = """The unanimous Declaration of the thirteen united States of America, When in the Course of human events, it becomes necessary for one people to dissolve the political bands which have connected them with another, and to assume among the powers of the earth, the separate and equal station to which the Laws of Nature and of Nature's God entitle them, a decent respect to the opinions of mankind requires that they should declare the causes which impel them to the separation.
We hold these truths to be self-evident, that all men are created equal, that they are endowed by their Creator with certain unalienable Rights, that among these are Life, Liberty and the pursuit of Happiness.--That to secure these rights, Governments are instituted among Men, deriving their just powers from the consent of the governed, --That whenever any Form of Government becomes destructive of these ends, it is the Right of the People to alter or to abolish it, and to institute new Government, laying its foundation on such principles and organizing its powers in such form, as to them shall seem most likely to effect their Safety and Happiness. Prudence, indeed, will dictate that Governments long established should not be changed for light and transient causes; and accordingly all experience hath shewn, that mankind are more disposed to suffer, while evils are sufferable, than to right themselves by abolishing the forms to which they are accustomed. But when a long train of abuses and usurpations, pursuing invariably the same Object evinces a design to reduce them under absolute Despotism, it is their right, it is their duty, to throw off such Government, and to provide new Guards for their future security.--Such has been the patient sufferance of these Colonies; and such is now the necessity which constrains them to alter their former Systems of Government. The history of the present King of Great Britain is a history of repeated injuries and usurpations, all having in direct object the establishment of an absolute Tyranny over these States. To prove this, let Facts be submitted to a candid world.
He has refused his Assent to Laws, the most wholesome and necessary for the public good.
He has forbidden his Governors to pass Laws of immediate and pressing importance, unless suspended in their operation till his Assent should be obtained; and when so suspended, he has utterly neglected to attend to them.
He has refused to pass other Laws for the accommodation of large districts of people, unless those people would relinquish the right of Representation in the Legislature, a right inestimable to them and formidable to tyrants only.
He has called together legislative bodies at places unusual, uncomfortable, and distant from the depository of their public Records, for the sole purpose of fatiguing them into compliance with his measures.
He has dissolved Representative Houses repeatedly, for opposing with manly firmness his invasions on the rights of the people.
He has refused for a long time, after such dissolutions, to cause others to be elected; whereby the Legislative powers, incapable of Annihilation, have returned to the People at large for their exercise; the State remaining in the mean time exposed to all the dangers of invasion from without, and convulsions within.
He has endeavoured to prevent the population of these States; for that purpose obstructing the Laws for Naturalization of Foreigners; refusing to pass others to encourage their migrations hither, and raising the conditions of new Appropriations of Lands.
He has obstructed the Administration of Justice, by refusing his Assent to Laws for establishing Judiciary powers.
He has made Judges dependent on his Will alone, for the tenure of their offices, and the amount and payment of their salaries.
He has erected a multitude of New Offices, and sent hither swarms of Officers to harrass our people, and eat out their substance.
He has kept among us, in times of peace, Standing Armies without the Consent of our legislatures.
He has affected to render the Military independent of and superior to the Civil power.
He has combined with others to subject us to a jurisdiction foreign to our constitution, and unacknowledged by our laws; giving his Assent to their Acts of pretended Legislation:
For Quartering large bodies of armed troops among us:
For protecting them, by a mock Trial, from punishment for any Murders which they should commit on the Inhabitants of these States:
For cutting off our Trade with all parts of the world:
For imposing Taxes on us without our Consent:
For depriving us in many cases, of the benefits of Trial by Jury:
For transporting us beyond Seas to be tried for pretended offences
For abolishing the free System of English Laws in a neighbouring Province, establishing therein an Arbitrary government, and enlarging its Boundaries so as to render it at once an example and fit instrument for introducing the same absolute rule into these Colonies:
For taking away our Charters, abolishing our most valuable Laws, and altering fundamentally the Forms of our Governments:
For suspending our own Legislatures, and declaring themselves invested with power to legislate for us in all cases whatsoever.
He has abdicated Government here, by declaring us out of his Protection and waging War against us.
He has plundered our seas, ravaged our Coasts, burnt our towns, and destroyed the lives of our people.
He is at this time transporting large Armies of foreign Mercenaries to compleat the works of death, desolation and tyranny, already begun with circumstances of Cruelty & perfidy scarcely paralleled in the most barbarous ages, and totally unworthy the Head of a civilized nation.
He has constrained our fellow Citizens taken Captive on the high Seas to bear Arms against their Country, to become the executioners of their friends and Brethren, or to fall themselves by their Hands.
He has excited domestic insurrections amongst us, and has endeavoured to bring on the inhabitants of our frontiers, the merciless Indian Savages, whose known rule of warfare, is an undistinguished destruction of all ages, sexes and conditions.
In every stage of these Oppressions We have Petitioned for Redress in the most humble terms: Our repeated Petitions have been answered only by repeated injury. A Prince whose character is thus marked by every act which may define a Tyrant, is unfit to be the ruler of a free people.
Nor have We been wanting in attentions to our Brittish brethren. We have warned them from time to time of attempts by their legislature to extend an unwarrantable jurisdiction over us. We have reminded them of the circumstances of our emigration and settlement here. We have appealed to their native justice and magnanimity, and we have conjured them by the ties of our common kindred to disavow these usurpations, which, would inevitably interrupt our connections and correspondence. They too have been deaf to the voice of justice and of consanguinity. We must, therefore, acquiesce in the necessity, which denounces our Separation, and hold them, as we hold the rest of mankind, Enemies in War, in Peace Friends.
We, therefore, the Representatives of the united States of America, in General Congress, Assembled, appealing to the Supreme Judge of the world for the rectitude of our intentions, do, in the Name, and by Authority of the good People of these Colonies, solemnly publish and declare, That these United Colonies are, and of Right ought to be Free and Independent States; that they are Absolved from all Allegiance to the British Crown, and that all political connection between them and the State of Great Britain, is and ought to be totally dissolved; and that as Free and Independent States, they have full Power to levy War, conclude Peace, contract Alliances, establish Commerce, and to do all other Acts and Things which Independent States may of right do. And for the support of this Declaration, with a firm reliance on the protection of divine Providence, we mutually pledge to each other our Lives, our Fortunes and our sacred Honor."""

In [118]:
#Processing the text data
import re

def text_cleaner(text):
    # lower case text
    newString = text.lower()
    newString = re.sub(r"'s\b","",newString)
    # remove punctuations
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    long_words=[]
    # remove short word
    for i in newString.split():
        if len(i)>=3:                  
            long_words.append(i)
    return (" ".join(long_words)).strip()

# preprocess the text
data_new = text_cleaner(data_text)

In [119]:
# creating sequences as the input data, so 30 characters as context and then predicting the next character
def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        # select sequence of tokens
        seq = text[i-length:i+1]
        # store
        sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences

# create sequences   
sequences = create_seq(data_new)
print(sequences)


Total Sequences: 7052
['the unanimous declaration the t', 'he unanimous declaration the th', 'e unanimous declaration the thi', ' unanimous declaration the thir', 'unanimous declaration the thirt', 'nanimous declaration the thirte', 'animous declaration the thirtee', 'nimous declaration the thirteen', 'imous declaration the thirteen ', 'mous declaration the thirteen u', 'ous declaration the thirteen un', 'us declaration the thirteen uni', 's declaration the thirteen unit', ' declaration the thirteen unite', 'declaration the thirteen united', 'eclaration the thirteen united ', 'claration the thirteen united s', 'laration the thirteen united st', 'aration the thirteen united sta', 'ration the thirteen united stat', 'ation the thirteen united state', 'tion the thirteen united states', 'ion the thirteen united states ', 'on the thirteen united states a', 'n the thirteen united states am', ' the thirteen united states ame', 'the thirteen united states amer', 'he thirteen united states ameri

In [120]:
# Encoding each character in sequence as numbers
# create a character mapping index
chars = sorted(list(set(data_new)))
mapping = dict((c, i) for i, c in enumerate(chars))

def encode_seq(seq):
    sequences = list()
    for line in seq:
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)
    return sequences

# encode the sequences
sequences = encode_seq(sequences)
print(sequences)

[[20, 8, 5, 0, 21, 14, 1, 14, 9, 13, 15, 21, 19, 0, 4, 5, 3, 12, 1, 18, 1, 20, 9, 15, 14, 0, 20, 8, 5, 0, 20], [8, 5, 0, 21, 14, 1, 14, 9, 13, 15, 21, 19, 0, 4, 5, 3, 12, 1, 18, 1, 20, 9, 15, 14, 0, 20, 8, 5, 0, 20, 8], [5, 0, 21, 14, 1, 14, 9, 13, 15, 21, 19, 0, 4, 5, 3, 12, 1, 18, 1, 20, 9, 15, 14, 0, 20, 8, 5, 0, 20, 8, 9], [0, 21, 14, 1, 14, 9, 13, 15, 21, 19, 0, 4, 5, 3, 12, 1, 18, 1, 20, 9, 15, 14, 0, 20, 8, 5, 0, 20, 8, 9, 18], [21, 14, 1, 14, 9, 13, 15, 21, 19, 0, 4, 5, 3, 12, 1, 18, 1, 20, 9, 15, 14, 0, 20, 8, 5, 0, 20, 8, 9, 18, 20], [14, 1, 14, 9, 13, 15, 21, 19, 0, 4, 5, 3, 12, 1, 18, 1, 20, 9, 15, 14, 0, 20, 8, 5, 0, 20, 8, 9, 18, 20, 5], [1, 14, 9, 13, 15, 21, 19, 0, 4, 5, 3, 12, 1, 18, 1, 20, 9, 15, 14, 0, 20, 8, 5, 0, 20, 8, 9, 18, 20, 5, 5], [14, 9, 13, 15, 21, 19, 0, 4, 5, 3, 12, 1, 18, 1, 20, 9, 15, 14, 0, 20, 8, 5, 0, 20, 8, 9, 18, 20, 5, 5, 14], [9, 13, 15, 21, 19, 0, 4, 5, 3, 12, 1, 18, 1, 20, 9, 15, 14, 0, 20, 8, 5, 0, 20, 8, 9, 18, 20, 5, 5, 14, 0], [13, 15, 21,

In [121]:
from sklearn.model_selection import train_test_split

# vocabulary size
vocab = len(mapping)
sequences = np.array(sequences)
# create X and y
X, y = sequences[:,:-1], sequences[:,-1]
# one hot encode y
y = to_categorical(y, num_classes=vocab)
# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Train shape: (6346, 30) Val shape: (706, 30)


In [122]:
#After creating training and validation sets above
# Building model now
# define model
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())

# compile the model
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
# fit the model
model.fit(X_tr, y_tr, epochs=100, verbose=2, validation_data=(X_val, y_val))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 50)            1350      
_________________________________________________________________
gru_1 (GRU)                  (None, 150)               90900     
_________________________________________________________________
dense_1 (Dense)              (None, 27)                4077      
Total params: 96,327
Trainable params: 96,327
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
199/199 - 21s - loss: 2.7400 - acc: 0.2064 - val_loss: 2.3861 - val_acc: 0.3144
Epoch 2/100
199/199 - 17s - loss: 2.2842 - acc: 0.3301 - val_loss: 2.2193 - val_acc: 0.3414
Epoch 3/100
199/199 - 18s - loss: 2.1503 - acc: 0.3566 - val_loss: 2.1339 - val_acc: 0.3725
Epoch 4/100
199/199 - 18s - loss: 2.0441 - acc: 0.3936 - val_loss: 2.0843 - val_acc: 0.4023
Epoch 5

In [136]:
#After training model above, lets use it predict
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		out_char = ''
		for char, index in mapping.items():
			if index == yhat:
				out_char = char
				break
		# append to input
		in_text += char
	return in_text

In [103]:
%ls

asdf/  CBTest.tgz  sample_data/


In [138]:
#Now lest call generate sequence
# import tarfile
 
# def extractTarFile(fileName):
#   if fileName.endswith("gz"):
#     tar = tarfile.open(fileName, "r:gz")
#     tar.extractall()
#     tar.close()
#   elif fileName.endswith("tar"):
#     tar = tarfile.open(fileName, "r:")
#     tar.extractall()
#     tar.close()


# extractTarFile("CBTest.tgz")

print(mapping)
print(generate_seq(model, mapping, 13, "imposing taxes", 20))

{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
imposing taxes without and consang


In [110]:
!pip install pytorch-transformers

     |████████████████████████████████| 184kB 12.1MB/s 
     |████████████████████████████████| 133kB 23.4MB/s 
     |████████████████████████████████| 1.1MB 23.0MB/s 
     |████████████████████████████████| 890kB 43.6MB/s 
     |████████████████████████████████| 6.9MB 47.9MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=a451e8141bd393eca968588040ecc0381f7fb8e14524d499da65b037313c3940
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.23 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [111]:
# Import required libraries
import torch
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Encode a text inputs
text = "What is the fastest car in the"
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])

# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the model in evaluation mode to deactivate the DropOut modules
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# Get the predicted next sub-word
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])

# Print the predicted word
print(predicted_text)

100%|██████████| 548118077/548118077 [00:17<00:00, 30495020.37B/s]


RuntimeError: ignored